In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
# https://github.com/tensorflow/tensorflow/issues/33721 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import seaborn as sns
import time
from csv import QUOTE_NONE
from ipywidgets import IntProgress

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
train_df = pd.read_csv('../sts-train.csv', sep='\t', usecols=[4, 5, 6], header=None, quoting=QUOTE_NONE, names=['score', 'sentence1', 'sentence2'])
validation_df = pd.read_csv('../sts-dev.csv', sep='\t', usecols=[4, 5, 6], header=None, quoting=QUOTE_NONE, names=['score', 'sentence1', 'sentence2'])
test_df = pd.read_csv('../sts-test.csv', sep='\t', usecols=[4, 5, 6], header=None, quoting=QUOTE_NONE, names=['score', 'sentence1', 'sentence2'])

In [5]:
train_df.head()

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [6]:
train_df = train_df.dropna()
validation_df = validation_df.dropna()
test_df = test_df.dropna()

In [7]:
train_df.count()

score        5749
sentence1    5749
sentence2    5749
dtype: int64

# Reuse pre-trained model

In [8]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = ["Incorrect", "Correct"]

In [9]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            truncation=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


In [10]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_outputs = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    last_hidden_state = bert_outputs[0]
    
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(last_hidden_state)
    dense_layer_1 = tf.keras.layers.Dense(32, activation="sigmoid")(avg_pool)
    dropout = tf.keras.layers.Dropout(0.5)(dense_layer_1)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/ta

In [11]:
# Unfreeze the bert_model.
bert_model.trainable = False
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer) 

In [12]:
model.load_weights('./pre-trained-bert.h5')

In [13]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    return {'Incorrect': proba[0], 'Correct': proba[1]}

In [14]:
sentence1 = 'A distributed system is a system from multiple computers which communicate with each other via network'
sentence2 = 'A distributed system is a system from one computer without network'
check_similarity(sentence1, sentence2)

/home/tr92346-local/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2073: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'Incorrect': 0.9974874, 'Correct': 0.0025125437}

# Create new model

In [15]:
len(model.layers)

8

In [16]:
model.layers.pop()

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_masks[0][0]        

In [18]:
with strategy.scope():
    dopout_layer = model.layers[-2].output
    output_layer = tf.keras.layers.Dense(1)(dopout_layer)
    model_regression = tf.keras.models.Model(inputs=model.input, outputs=output_layer)
    model_regression.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="mse",
        metrics=[
            tf.keras.metrics.RootMeanSquaredError()
        ],
    )
    
model_regression.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________

# Train new model based on new dataset

In [19]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self,
        sentence_pairs,
        scores,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.scores = scores
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            truncation=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            scores = np.array(self.scores[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], scores
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [20]:
train_data = BertSemanticDataGenerator(
    train_df[["sentence1", "sentence2"]].values.astype("str"),
    train_df['score'].values,
    batch_size=batch_size,
    shuffle=True,
)

valid_data = BertSemanticDataGenerator(
    validation_df[["sentence1", "sentence2"]].values.astype("str"),
    validation_df['score'].values,
    batch_size=batch_size,
    shuffle=False,
)

test_data = BertSemanticDataGenerator(
    test_df[["sentence1", "sentence2"]].values.astype("str"),
    test_df['score'].values,
    batch_size=batch_size,
    shuffle=False,
)

In [21]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [22]:
time_callback = TimeHistory()

In [23]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 25

In [24]:
history = model_regression.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
    callbacks = [time_callback]
)

Epoch 1/25
179/179 [==============================] - 47s 214ms/step - loss: 4.4857 - root_mean_squared_error: 2.0957 - val_loss: 1.5341 - val_root_mean_squared_error: 1.2386
Epoch 2/25
179/179 [==============================] - 37s 205ms/step - loss: 1.7279 - root_mean_squared_error: 1.3143 - val_loss: 1.3753 - val_root_mean_squared_error: 1.1727
Epoch 3/25
179/179 [==============================] - 37s 208ms/step - loss: 1.5808 - root_mean_squared_error: 1.2572 - val_loss: 1.2726 - val_root_mean_squared_error: 1.1281
Epoch 4/25
179/179 [==============================] - 37s 209ms/step - loss: 1.4928 - root_mean_squared_error: 1.2217 - val_loss: 1.2270 - val_root_mean_squared_error: 1.1077
Epoch 5/25
179/179 [==============================] - 37s 209ms/step - loss: 1.4169 - root_mean_squared_error: 1.1902 - val_loss: 1.1298 - val_root_mean_squared_error: 1.0629
Epoch 6/25
179/179 [==============================] - 37s 209ms/step - loss: 1.3698 - root_mean_squared_error: 1.1701 - val_l

In [25]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'custom-bert-history-03.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [26]:
print(time_callback.times)

[47.34685039520264, 36.747371673583984, 37.2716429233551, 37.30723977088928, 37.32818961143494, 37.38395857810974, 37.40026879310608, 37.443514585494995, 37.42617201805115, 37.43510365486145, 37.43345260620117, 37.425671100616455, 37.48271059989929, 37.45222282409668, 37.406033992767334, 37.356741189956665, 37.43113970756531, 37.48248052597046, 37.43406963348389, 37.41909050941467, 37.379101037979126, 37.32138657569885, 37.44663906097412, 37.399582862854004, 37.39009094238281]


In [27]:
hist_time_df = pd.DataFrame(time_callback.times) 
hist_csv_file = 'custom-bert-time-03.csv'
with open(hist_csv_file, mode='w') as f:
    hist_time_df.to_csv(f)

In [28]:
model_regression.save_weights('./custom-bert-03.h5')

# Fine-tuning

In [29]:
with strategy.scope():
    # Unfreeze the bert_model.
    bert_model.trainable = True
    # Recompile the model to make the change effective.
    model_regression.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss="mse",
        metrics=[
            tf.keras.metrics.RootMeanSquaredError()
        ],
    )

model_regression.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_masks[0][0]      

In [30]:
time_callback = TimeHistory()

In [31]:
history = model_regression.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
    callbacks = [time_callback]
)

Epoch 1/25
179/179 [==============================] - 100s 497ms/step - loss: 0.9671 - root_mean_squared_error: 0.9828 - val_loss: 0.6244 - val_root_mean_squared_error: 0.7902
Epoch 2/25
179/179 [==============================] - 87s 487ms/step - loss: 0.6959 - root_mean_squared_error: 0.8341 - val_loss: 0.5953 - val_root_mean_squared_error: 0.7715
Epoch 3/25
179/179 [==============================] - 87s 487ms/step - loss: 0.5775 - root_mean_squared_error: 0.7598 - val_loss: 0.5625 - val_root_mean_squared_error: 0.7500
Epoch 4/25
179/179 [==============================] - 87s 486ms/step - loss: 0.5002 - root_mean_squared_error: 0.7072 - val_loss: 0.5451 - val_root_mean_squared_error: 0.7383
Epoch 5/25
179/179 [==============================] - 87s 484ms/step - loss: 0.4788 - root_mean_squared_error: 0.6918 - val_loss: 0.5509 - val_root_mean_squared_error: 0.7422
Epoch 6/25
179/179 [==============================] - 86s 483ms/step - loss: 0.4340 - root_mean_squared_error: 0.6587 - val_

In [32]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'custom-bert-fine-tuned-history-03.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [33]:
hist_time_df = pd.DataFrame(time_callback.times) 
hist_csv_file = 'custom-bert-fine-tuned-time-03.csv'
with open(hist_csv_file, mode='w') as f:
    hist_time_df.to_csv(f)

In [34]:
model_regression.save_weights('./custom-bert-fine-tuned-03.h5')

# Evaluation

In [35]:
model_regression.evaluate(test_data, verbose=1)

43/43 [==============================] - 7s 152ms/step - loss: 0.6430 - root_mean_squared_error: 0.8019


[0.6430194973945618, 0.8018849492073059]

# Testing

In [36]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    data = BertSemanticDataGenerator(
        sentence_pairs, scores=None, batch_size=1, shuffle=False, include_targets=False,
    )

    score = model_regression.predict(data)[0]
    return score

In [37]:
sentence1 = 'A distributed system is a system from multiple computers which communicate with each other via network'
sentence2 = 'A distributed system is a system from one computer without network'
check_similarity(sentence1, sentence2)

array([2.0084372], dtype=float32)

In [38]:
sentence1 = 'asdsd'
sentence2 = 'asdsad are good sdsdsdsds'
check_similarity(sentence1, sentence2)

array([2.0672293], dtype=float32)

In [39]:
sentence1 = 'the activity of learning or being trained'
sentence2 = 'the gradual process of acquiring knowledge.'
check_similarity(sentence1, sentence2)

array([2.0244303], dtype=float32)